## Load Open Web UI Token

In [10]:
import yaml
with open('conf.yaml', 'r') as file:
    conf = yaml.safe_load(file)
token = conf["WEB_UI_KEY"]

## Connects to OpenWebUI (RAGaRenn)

In [11]:
from data_boost import WebUIConnector, create_acronym_prompt

url = "https://llmrns.rp.eskn.fr/marius.garenaux-gruau@irisa.fr/api/chat/completions"
owui = WebUIConnector(token, url)

## Creates custom prompt and asks RAGaRenn

In [12]:
import json

n_conv = 10
with open("./data/raw_data.json", "rt") as f:
    all_acro = json.load(f)
n_acros = len(all_acro)

In [13]:
def create_conv(all_acro: list, n_batch):
    boosted_convs = []
    n_acros = len(all_acro)
    print(f"Batch number {n_batch}")
    for k, each_elem in enumerate(all_acro):
        
        acro = each_elem['acro']
        acro_def = each_elem["definition"] if "definition" in each_elem else None
        acro_def_verbose = each_elem["verbose_def"] if "verbose_def" in each_elem else None

        prompt = create_acronym_prompt(n_conv, acro, acro_def, acro_def_verbose)
        answer = owui.get_chat_response(prompt, return_list=True)
        if answer is None or answer == []:
            continue
        boosted_convs += answer
        # boosted_convs += [[
        #     {
        #         "role": "user",
        #         "content": acro
        #     },
        #     {
        #         "role": "assistant",
        #         "content": acro_def
        #     }
        # ]]
        print(k+1, "on", n_acros)

    with open(f"./data/boosted_data_{n_batch}.json", "wt") as f:
        json.dump(boosted_convs, f, indent=4)
    

In [14]:
batch_size = 5
n_batch = n_acros//batch_size
start_on_batch = 2
for k in range(n_batch+1):
    if k < start_on_batch:
        continue
    start_batch = k*batch_size
    end_batch = (k+1)*batch_size
    data = all_acro[start_batch:end_batch]
    create_conv(data, k)

Batch number 2
1 on 5
2 on 5
Error: 504 <html><body><h1>504 Gateway Time-out</h1>
The server didn't respond in time.
</body></html>


4 on 5
Error: 504 <html><body><h1>504 Gateway Time-out</h1>
The server didn't respond in time.
</body></html>


Batch number 3
Error during parsing result of request to json, skipping result.
Error during parsing result of request to json, skipping result.
3 on 5
Error: 504 <html><body><h1>504 Gateway Time-out</h1>
The server didn't respond in time.
</body></html>


5 on 5
Batch number 4
1 on 5
Error: 504 <html><body><h1>504 Gateway Time-out</h1>
The server didn't respond in time.
</body></html>


3 on 5
4 on 5
Error during parsing result of request to json, skipping result.
Batch number 5
Error: 504 <html><body><h1>504 Gateway Time-out</h1>
The server didn't respond in time.
</body></html>


2 on 5
3 on 5
4 on 5
5 on 5
Batch number 6
1 on 5
2 on 5
3 on 5
4 on 5
5 on 5
Batch number 7
1 on 5
Error during parsing result of request to json, skipping result.

In [15]:
# Merge all batches
n_batches = 57
all_data = []
for i in range(n_batches):
    with open(f"./data/boosted_data_{i}.json", "rt") as f:
        d = json.load(f)
    all_data += d

In [17]:
with open("./data/boosted_data.json", "wt") as f:
    json.dump(all_data, f)